# Translation API `v0.0.5` Examples

This notebook uses the `requests` library to interact with the version `v0.0.5` of the Translation API. It demonstrates how to send text for translation and receive the translated output.

**Note**: This notebook only covers the parts of the API that have changed since version `v0.0.1`. For a complete exploration of the API's features, please refer to the `api_v1.ipynb` notebook in the same directory.

## Table of Contents

1. [Root Endpoint](#1.-Root-Endpoint)
2. [Prediction Endpoint](#2.-Prediction-Endpoint)
   - [2.1 Single Translation with pre-loaded model and basic parameters](#2.1-Single-Translation-with-pre-loaded-model-and-basic-parameters)
   - [2.2 Response to unavailable model request](#2.2-Response-to-unavailable-model-request)

In [1]:
# get required libraries
import json
import time
import requests

# define constants
base_api_url = 'http://0.0.0.0:8000/'

## 1. Root Endpoint

The root endpoint (`/`) provides basic information about the API.

In [2]:
start_time = time.time()
response = requests.get(base_api_url)
end_time = time.time()

print("Request time (s):", round(end_time - start_time, 3))
print("Response status code:", response.status_code)
try:
    content = json.loads(response.content)
    print("Response content:", json.dumps(content, indent=4))
except json.JSONDecodeError:
    print("Response content is not valid JSON:", response.content)

Request time (s): 0.027
Response status code: 200
Response content: {
    "name": "Translation API",
    "version": "v0.0.5",
    "description": "API for text translation using pre-trained Transformer models."
}


## 2. Prediction Endpoint

On `v0.0.5` of the API, each translation pair has its own dedicated enpoint of the form `/predict/{source}-{target}`, and it is no longer possible to request translations for different language pair in a single request.

### 2.1 Single Translation with pre-loaded model and basic parameters

This first example shows how to perform a single translation to a pre-loaded model using only the required parameter: `text`.

In [2]:
translation_pair = 'en-es'
base_prediction_url = '{base_api_url}predict/{translation_pair}/'

payload = {
    "items": [{
        "text": "The boy with fair hair lowered himself down in the last few feet of rocky ground and began to pick his way toward the lagoon."
    }]
}

start_time = time.time()
response = requests.post(
    base_prediction_url.format(
        base_api_url=base_api_url,
        translation_pair=translation_pair
    ),
    json=payload
)
end_time = time.time()

print("Request time (s):", round(end_time - start_time, 3))
print("Response status code:", response.status_code)
try:
    content = json.loads(response.content)
    print("Response content:", json.dumps(content, indent=4))
except json.JSONDecodeError:
    print("Response content is not valid JSON:", response.content)

Request time (s): 2.008
Response status code: 200
Response content: {
    "results": [
        {
            "position": 0,
            "result": "El chico de pelo limpio se baj\u00f3 en los \u00faltimos pies de tierra rocosa y comenz\u00f3 a abrirse camino hacia la laguna."
        }
    ]
}


Let's try translating the same sentence to French.

In [3]:
translation_pair = 'en-fr'

start_time = time.time()
response = requests.post(
    base_prediction_url.format(
        base_api_url=base_api_url,
        translation_pair=translation_pair
    ),
    json=payload
)
end_time = time.time()

print("Request time (s):", round(end_time - start_time, 3))
print("Response status code:", response.status_code)
try:
    content = json.loads(response.content)
    print("Response content:", json.dumps(content, indent=4))
except json.JSONDecodeError:
    print("Response content is not valid JSON:", response.content)

Request time (s): 1.566
Response status code: 200
Response content: {
    "results": [
        {
            "position": 0,
            "result": "Le gar\u00e7on aux cheveux justes s'est abaiss\u00e9 dans les derniers pieds de terre rocheuse et a commenc\u00e9 \u00e0 prendre son chemin vers la lagune."
        }
    ]
}


The rest of the content remains unchanged, the API can still process batch requests and advanced parameters as before.

### 2.2 Response to unavailable model request

The `predict/` endpoint performs an initial validation to check if the requested model is available. If the model is not found, it returns a `422` error with a descriptive message.

In [4]:
translation_pair = 'elvish-sindarin'

start_time = time.time()
response = requests.post(
    base_prediction_url.format(
        base_api_url=base_api_url,
        translation_pair=translation_pair
    ),
    json=payload
)
end_time = time.time()

print("Request time (s):", round(end_time - start_time, 3))
print("Response status code:", response.status_code)
try:
    content = json.loads(response.content)
    print("Response content:", json.dumps(content, indent=4))
except json.JSONDecodeError:
    print("Response content is not valid JSON:", response.content)

Request time (s): 0.014
Response status code: 422
Response content: {
    "detail": "Translation pair 'elvish-sindarin' is not supported.Available pairs: ['en-fr', 'en-es', 'es-en', 'en-de', 'de-en', 'fr-es']"
}
